In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchmetrics as TM
pl.utilities.seed.seed_everything(seed=42)

Global seed set to 42


42

In [ ]:


import pandas as pd
import sys, os
source_path = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)

from models import NeuralNetwork, Trainer
from preprocessing import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader)

Global seed set to 42


# Utility functions and classes

# Set Root Path to local disk

In [ ]:
ROOT_PATH = 'c:/Users/gilbe/Documents/TokyoData'

# Start data analysis 

### API data

In [ ]:
'/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv'
'/train_files/trades.csv'

train_df = pd.read_csv(f'{ROOT_PATH}/train_files/stock_prices.csv')
train_df['Date'] = pd.to_datetime(train_df['Date']) 
train_df.set_index('Date', inplace=True)
# train_df = date_features(train_df)

train_options = pd.read_csv(f'{ROOT_PATH}/train_files/options.csv', low_memory=False)
train_financials = pd.read_csv(f'{ROOT_PATH}/train_files/financials.csv', low_memory=False)
train_trades = pd.read_csv(f'{ROOT_PATH}/train_files/trades.csv', low_memory=False)

In [ ]:
print(train_df.shape)
print()
print('Unique values for Adjustment factor:')
print(train_df.AdjustmentFactor.unique())
print()
print('Number of Unique Securities code:')
print(train_df.SecuritiesCode.nunique())
print()
print('Number of Unique Expected dividends:')
print(train_df.ExpectedDividend.nunique())
# print(train_df.ExpectedDividend.unique())
show_df(train_df, dataframe_name='stock_data')

show_df(train_options, dataframe_name='options_data')

show_df(train_financials, dataframe_name='financials')

show_df(train_trades, dataframe_name='trades')

In [ ]:
import matplotlib.pyplot as plt

df_1301 = train_df[train_df['SecuritiesCode'] == 1301].drop('SecuritiesCode', axis=1)
# df_1301.set_index('Date', inplace=True)
# df_1301[['Open', 'Close', 'High', 'Low', 'Volume']] = df_1301[['Open', 'Close', 'High', 'Low', 'Volume']].pct_change()
df_1301 = preprocess(df_1301)
df_1301.plot(figsize=(15, 20), subplots=True);
plt.show();
df_1301.Target.hist(bins=50);
plt.show();


# pd.plotting.scatter_matrix(df_1301);
# plt.show();
print(df_1301.info())

## Models
Todo:

    1) Preprocesses data and use correct features (date features, etc)
    2) Deep learning for prediction of stock-returns
    3) Ranking with XGBoost or other methods

In [ ]:
import torch
ytrain_stock = df_1301['Target'].to_numpy()
xtrain_stock = df_1301.drop('Target', axis=1).to_numpy()
print(ytrain_stock)
train_dataloader = get_loader(xtrain_stock, ytrain_stock, 64)


# train_set = TimeSeriesDataset(x_num, y_num)
# train_loader = DataLoader(train_set, batch_size=64)
model = NeuralNetwork(in_features=xtrain_stock.shape[1], out_features=1)
trainer = Trainer(model)


# # Display image and label.
# train_features, train_labels = next(iter(train_dataloader))
# print(train_features['features'])
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# # img = train_features[0].squeeze()
# # label = train_labels[0]
# # plt.imshow(img, cmap="gray")
# # plt.show()
# # print(f"Label: {label}")